In [6]:
from transformers import BertTokenizer
# 토크나이져 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
sentences = [
    'Hello, world!',
    'unbelievable performance!',
    'COVID-19 pandemic'
]
for sentence in sentences:
   # 토큰화
   tokens =  tokenizer.tokenize(sentence)
   print(f'원문 : {sentence}')
   print(f'토큰 : {tokens}')

   #ID 변환
   ids = tokenizer.convert_tokens_to_ids(tokens)
   print(f'ID : {ids}')

   # 역변환
   decoded_string = tokenizer.decode(ids)
   print(f'역변환 : {decoded_string}\n')

원문 : Hello, world!
토큰 : ['hello', ',', 'world', '!']
ID : [7592, 1010, 2088, 999]
역변환 : hello, world!

원문 : unbelievable performance!
토큰 : ['unbelievable', 'performance', '!']
ID : [23653, 2836, 999]
역변환 : unbelievable performance!

원문 : COVID-19 pandemic
토큰 : ['co', '##vid', '-', '19', 'pan', '##de', '##mic']
ID : [2522, 17258, 1011, 2539, 6090, 3207, 7712]
역변환 : covid - 19 pandemic



In [7]:
# 2. Attention Mask : 실제단어 1  , 패딩은 0
from transformers import BertTokenizer
# 토크나이져 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
sentences = [
    'short sentence',
    'Thiis is a much longer sentence with more words'
]
# 여러문장을 한꺼번에 토크나이징하고 가장 긴 문장길이에 맞춰 자동 패딩 수행
encoded = tokenizer(
    sentences,
    padding=True,
    return_tensors='pt'
)
encoded


{'input_ids': tensor([[  101,  2460,  6251,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [  101, 16215,  6137,  2015,  2003,  1037,  2172,  2936,  6251,  2007,
          2062,  2616,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [18]:
# token_type_ids : 두 문장을 입력할때 첫번째 ,두번째 구분
from transformers import BertTokenizer
# 토크나이져 로드
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
sentence_A = "The weather is nice"
sentence_B = "Let's go for a walk"
# 두 문장을 하나의 입력으로 인코딩
encoded = tokenizer(
    sentence_A,
    sentence_B,
    padding=True,
    return_tensors="pt"
)
tokens = tokenizer.convert_ids_to_tokens(encoded["input_ids"][0])
for token,token_id, type_id in zip(tokens, encoded["input_ids"][0],encoded["token_type_ids"][0]):
  segment = "문장 A" if type_id == 0 else "문장 B"
  if token == "[SEP]":
    segment = "구분자"
  elif token == "[CLS]":
    segment = "시작"
  print(f'{token:20s} {token_id.item():6d} {type_id.item():6d} ({segment})')

[CLS]                   101      0 (시작)
the                    1996      0 (문장 A)
weather                4633      0 (문장 A)
is                     2003      0 (문장 A)
nice                   3835      0 (문장 A)
[SEP]                   102      0 (구분자)
let                    2292      1 (문장 B)
'                      1005      1 (문장 B)
s                      1055      1 (문장 B)
go                     2175      1 (문장 B)
for                    2005      1 (문장 B)
a                      1037      1 (문장 B)
walk                   3328      1 (문장 B)
[SEP]                   102      1 (구분자)


In [22]:
# [CLS] Token Pooling : BERT 첫번째 토큰 [CLS] 문서 전체의 요약 => 분류 작업을 할때
# 이 토큰의 출력만 가져와서 분류기(classifier)에 연결
import torch
from transformers import BertTokenizer, BertModel
# 토크나이져 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
sentence = "BERT is amazing for NLP tasks!"
# 인코딩
inputs =  tokenizer(sentence,return_tensors='pt')
# BERT 통과
with torch.no_grad():
  outputs = model(**inputs)
#   출력 형태 확인
last_hidden_state = outputs.last_hidden_state
print(f'입력문장 : {sentence}')
print(f'last_hidden_state 형태 : {last_hidden_state.shape}')
print(f'batch_size = 1 sequence_length = {last_hidden_state.shape[1]} \
  hidden_size = {last_hidden_state.shape[2]}')
# [CLS]토큰 추출
cls_embedding = last_hidden_state[:, 0, :]
print(f'cls_embedding 형태 : {cls_embedding.shape}')
# 분류기 (2-class)
classifier = torch.nn.Linear(768,2)
logits = classifier(cls_embedding)
probs = torch.softmax(logits, dim=-1)
print(f'logits : {logits}')
print(f'probs : {probs}')
print(f'predicted class : {torch.argmax(probs).item()}')


입력문장 : BERT is amazing for NLP tasks!
last_hidden_state 형태 : torch.Size([1, 10, 768])
batch_size = 1 sequence_length = 10   hidden_size = 768
cls_embedding 형태 : torch.Size([1, 768])
logits : tensor([[-0.4586, -0.2816]], grad_fn=<AddmmBackward0>)
probs : tensor([[0.4559, 0.5441]], grad_fn=<SoftmaxBackward0>)
predicted class : 1
